# Timeline

<img src="https://i.ibb.co/KKVVDjW/Image-Tittle.jpg" alt="Image-Tittle" border="0">

<img src="https://i.ibb.co/HKtKPy6/announcments.jpg" alt="announcments" border="0">

# Declaration

* According to MoHFW https://www.mohfw.gov.in/ some new cases have been reassigned states as per latest information
* Confirmed cases includes both Indian and Foriegn citizen
* Cured cases includes cured, discharged and migrated patients 

## Data Source
* https://www.mohfw.gov.in/


### GitHub Repository of this Notebook
<a href="https://github.com/sreyaz01/covid-19-india-data-analysis"><img src="https://i.ibb.co/B3vvzTy/0-s-Y-XTIBzlfd2zskq.png" alt="0-s-Y-XTIBzlfd2zskq" border="0" width="300" height="100"></a>

In [33]:
#!conda install -c conda-forge cufflinks-py

# Libraries

In [34]:
## utility libraries
from IPython.core.display import HTML
from datetime import datetime
from datetime import timedelta


# storing and anaysis
import pandas as pd
import geopandas as gpd
import numpy as np

#Visualization Libraries
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import style

import plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.offline as pyo
from plotly.offline import init_notebook_mode,plot,iplot

import folium
import seaborn as sns

import cufflinks as cf

# Warning
import warnings
warnings.filterwarnings('ignore')


print('Pandas Version' , pd.__version__)
print('Matplotlib Version' , matplotlib.__version__)
print('Plotly Version' , plotly.__version__)
print('Seaborn Version' , sns.__version__)

Pandas Version 1.0.1
Matplotlib Version 3.2.1
Plotly Version 4.5.4
Seaborn Version 0.9.0


In [35]:
# setting up some setting for libraries
%matplotlib inline
plt.rcParams['figure.figsize'] = 17,8
pyo.init_notebook_mode(connected=True)
cf.go_offline()

#style.use('ggplot')

In [36]:
# color pallette
cnf = '#393e46' # confirmed - grey
dth = '#ff2e63' # death - red
rec = '#21bf73' # recovered - cyan
act = '#fe9801' # active case - yellow

# Dataset 

In [37]:
%ls C:\Users\sreya\Desktop\Pythone_File\COVID-19_INDIA\input\covid19-corona-virus-india-dataset

 Volume in drive C has no label.
 Volume Serial Number is 0821-B132

 Directory of C:\Users\sreya\Desktop\Pythone_File\COVID-19_INDIA\input\covid19-corona-virus-india-dataset

19-04-2020  11:47    <DIR>          .
19-04-2020  11:47    <DIR>          ..
10-04-2020  16:57    <DIR>          .ipynb_checkpoints
18-04-2020  23:04             7,749 cases_over_time_florish.csv
19-04-2020  11:44            54,684 complete.csv
10-04-2020  16:59            39,293 patient_wise_data_scrapping_and_cleaning.ipynb
10-04-2020  16:57         1,219,997 patients_data.csv
               4 File(s)      1,321,723 bytes
               3 Dir(s)  220,257,288,192 bytes free


In [38]:
#importing data
df = pd.read_csv(r'C:\Users\sreya\Desktop\Pythone_File\COVID-19_INDIA\input\covid19-corona-virus-india-dataset\complete.csv',
                parse_dates = ['Date'])

df.tail()

,Date,Name of State / UT,Total Confirmed cases (Indian National),Total Confirmed cases ( Foreign National ),Cured/Discharged/Migrated,Latitude,Longitude,Death,Total Confirmed cases
1074,2020-04-19,Telengana,0,0,186,18.1124,79.0193,18,809
1075,2020-04-19,Tripura,0,0,1,23.9408,91.9882,0,2
1076,2020-04-19,Uttar Pradesh,0,0,86,26.8467,80.9462,14,969
1077,2020-04-19,Uttarakhand,0,0,9,30.0668,79.0193,0,42
1078,2020-04-19,West Bengal,0,0,62,22.9868,87.8550,12,310


In [39]:
df.columns

Index(['Date', 'Name of State / UT', 'Total Confirmed cases (Indian National)',
       'Total Confirmed cases ( Foreign National )',
       'Cured/Discharged/Migrated', 'Latitude', 'Longitude', 'Death',
       'Total Confirmed cases'],
      dtype='object')

In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1079 entries, 0 to 1078
Data columns (total 9 columns):
 #   Column                                      Non-Null Count  Dtype         
---  ------                                      --------------  -----         
 0   Date                                        1079 non-null   datetime64[ns]
 1   Name of State / UT                          1079 non-null   object        
 2   Total Confirmed cases (Indian National)     1079 non-null   int64         
 3   Total Confirmed cases ( Foreign National )  1079 non-null   int64         
 4   Cured/Discharged/Migrated                   1079 non-null   int64         
 5   Latitude                                    1079 non-null   float64       
 6   Longitude                                   1079 non-null   float64       
 7   Death                                       1079 non-null   int64         
 8   Total Confirmed cases                       1079 non-null   int64         
dtypes: dateti

## State Map data

In [41]:
# shape files
map_data = gpd.read_file(r'C:\Users\sreya\Desktop\Pythone_File\COVID-19_INDIA\input\india-district-wise-shape-files\output.shp')

map_data.head()

,objectid,statecode,statename,state_ut,distcode,distname,distarea,totalpopul,totalhh,totpopmale,totpopfema,st_areasha,st_lengths,geometry
0,1,05,Uttarakhand,STATE,066,Nainital,4251.0,954605.0,191383.0,493666.0,460939.0,5.322546e+09,506182.695952,"POLYGON ((79.52659 29.05543, 79.52550 29.05545..."
1,2,05,Uttarakhand,STATE,060,Dehradun,3088.0,1696694.0,347001.0,892199.0,804495.0,4.177236e+09,578188.681639,"POLYGON ((77.87557 30.26052, 77.87467 30.26087..."
2,3,05,Uttarakhand,STATE,064,Almora,3144.0,622506.0,140577.0,291081.0,331425.0,4.140751e+09,463454.225766,"POLYGON ((79.28494 29.92735, 79.28495 29.92723..."
3,4,05,Uttarakhand,STATE,065,Champawat,1766.0,259648.0,53953.0,131125.0,128523.0,2.294297e+09,314508.010612,"POLYGON ((80.12479 29.01308, 80.12481 29.01306..."
4,5,05,Uttarakhand,STATE,056,Uttarkashi,8016.0,330086.0,67602.0,168597.0,161489.0,1.085166e+10,786425.588972,"POLYGON ((78.92267 31.25333, 78.93106 31.26840..."


In [42]:
#State Wise Grouping Data
states = map_data.dissolve(by='statename').reset_index()
states.head()

,statename,geometry,objectid,statecode,state_ut,distcode,distname,distarea,totalpopul,totalhh,totpopmale,totpopfema,st_areasha,st_lengths
0,Andaman & Nicobar Islands,"MULTIPOLYGON (((93.84419 7.22173, 93.84565 7.2...",380,35,UNION TERRITORY,639,North & Middle Andaman,3736.0,105597.0,26199.0,54861.0,50736.0,3.582828e+09,1.665121e+06
1,Andhra Pradesh,"MULTIPOLYGON (((82.36333 16.71061, 82.35167 16...",134,28,STATE,549,Prakasam,17626.0,3397448.0,860463.0,1714764.0,1682684.0,1.923605e+10,8.177556e+05
2,Arunachal Pradesh,"POLYGON ((95.48591 26.75778, 95.48534 26.75821...",404,12,STATE,259,Lohit,5212.0,145726.0,30005.0,76221.0,69505.0,5.238221e+09,4.149939e+05
3,Assam,"POLYGON ((89.83741 26.41240, 89.83771 26.41230...",420,18,STATE,302,Goalpara,1824.0,1008183.0,198454.0,513292.0,494891.0,2.452588e+09,3.629872e+05
4,Bihar,"POLYGON ((85.41963 24.54472, 85.41772 24.54562...",224,10,STATE,228,Sheikhpura,689.0,636342.0,100701.0,329743.0,306599.0,6.371338e+08,1.976915e+05


In [43]:
states['statename'] = states['statename'].str.replace('&', 'and')
states['statename'] = states['statename'].str.replace('NCT of ', '')
states['statename'] = states['statename'].str.replace('Chhatisgarh', 'Chhattisgarh')
states['statename'] = states['statename'].str.replace('Orissa', 'Odisha')
states['statename'] = states['statename'].str.replace('Pondicherry', 'Puducherry')

states['statename'].unique()

array(['Andaman and Nicobar Islands', 'Andhra Pradesh',
       'Arunachal Pradesh', 'Assam', 'Bihar', 'Chandigarh',
       'Chhattisgarh', 'Dadra and Nagar Haveli', 'Daman and Diu', 'Goa',
       'Gujarat', 'Haryana', 'Himachal Pradesh', 'Jammu and Kashmir',
       'Jharkhand', 'Karnataka', 'Kerala', 'Ladakh', 'Lakshadweep',
       'Madhya Pradesh', 'Maharashtra', 'Manipur', 'Meghalaya', 'Mizoram',
       'Delhi', 'Nagaland', 'Odisha', 'Puducherry', 'Punjab', 'Rajasthan',
       'Sikkim', 'Tamil Nadu', 'Telangana', 'Tripura', 'Uttar Pradesh',
       'Uttarakhand', 'West Bengal'], dtype=object)

# Preprocessing


## Cleaning

In [44]:
df_clean = df[['Date', 'Name of State / UT', 'Latitude', 'Longitude', 'Total Confirmed cases', 'Death', 'Cured/Discharged/Migrated']]
df_clean.columns = ['Date', 'State/UT', 'Latitude', 'Longitude', 'Confirmed', 'Deaths', 'Cured']

df_clean['Date'] = df_clean['Date'].dt.date

df_clean['Active'] = df_clean['Confirmed']-(df_clean['Deaths']+df_clean['Cured'])
df_clean['Mortality Rate'] = df_clean['Deaths']/df_clean['Confirmed']
df_clean['Recovery Rate'] = df_clean['Cured']/df_clean['Confirmed']
df_clean.tail()

,Date,State/UT,Latitude,Longitude,Confirmed,Deaths,Cured,Active,Mortality Rate,Recovery Rate
1074,2020-04-19,Telengana,18.1124,79.0193,809,18,186,605,0.022250,0.229913
1075,2020-04-19,Tripura,23.9408,91.9882,2,0,1,1,0.000000,0.500000
1076,2020-04-19,Uttar Pradesh,26.8467,80.9462,969,14,86,869,0.014448,0.088751
1077,2020-04-19,Uttarakhand,30.0668,79.0193,42,0,9,33,0.000000,0.214286
1078,2020-04-19,West Bengal,22.9868,87.8550,310,12,62,236,0.038710,0.200000


## Finding Latest Data Only

In [45]:
latest = df_clean[df_clean['Date']==max(df_clean['Date'])]


total_confirm = latest['Confirmed'].sum()
total_active = latest['Active'].sum()
total_cured = latest['Cured'].sum()
total_death = latest['Deaths'].sum()


now  = datetime.now().strftime("%B %d, %Y")

print(u"\u2022",f'Total Number of Confirmed Covid 2019 Cases across India till date ({now}):', total_confirm)
print(u"\u2022",f'Total Number of Active Cases till date ({now}):', total_active)
print(u"\u2022",f'Total Number of Cured Cases across India till date ({now}):', total_cured)
print(u"\u2022",f'Total Number of Cured Cases across India till date ({now}):', total_death)

• Total Number of Confirmed Covid 2019 Cases across India till date (April 19, 2020): 15712
• Total Number of Active Cases till date (April 19, 2020): 12974
• Total Number of Cured Cases across India till date (April 19, 2020): 2231
• Total Number of Cured Cases across India till date (April 19, 2020): 507


In [46]:
tm = latest.melt(id_vars="Date", value_vars=['Active', 'Deaths', 'Cured'])
tm.head()

,Date,variable,value
0,2020-04-19,Active,3
1,2020-04-19,Active,546
2,2020-04-19,Active,1
3,2020-04-19,Active,22
4,2020-04-19,Active,47


In [47]:
fig = px.treemap(tm, path=["variable"], values="value",height=250, width=800,
                 color_discrete_sequence=[act, rec, dth], title='Latest Stats')

fig.data[0].textinfo = 'label+value+text'
fig.show()

In [48]:
latest.head()

,Date,State/UT,Latitude,Longitude,Confirmed,Deaths,Cured,Active,Mortality Rate,Recovery Rate
1047,2020-04-19,Andaman and Nicobar Islands,11.7401,92.6586,14,0,11,3,0.000000,0.785714
1048,2020-04-19,Andhra Pradesh,15.9129,79.7400,603,15,42,546,0.024876,0.069652
1049,2020-04-19,Arunachal Pradesh,28.2180,94.7278,1,0,0,1,0.000000,0.000000
1050,2020-04-19,Assam,26.2006,92.9376,35,1,12,22,0.028571,0.342857
1051,2020-04-19,Bihar,25.0961,85.3131,86,2,37,47,0.023256,0.430233


In [49]:
temp = latest.groupby(by = ['State/UT']).sum()

temp.tail()

,Latitude,Longitude,Confirmed,Deaths,Cured,Active,Mortality Rate,Recovery Rate
State/UT,,,,,,,,
Telengana,18.1124,79.0193,809,18,186,605,0.022250,0.229913
Tripura,23.9408,91.9882,2,0,1,1,0.000000,0.500000
Uttar Pradesh,26.8467,80.9462,969,14,86,869,0.014448,0.088751
Uttarakhand,30.0668,79.0193,42,0,9,33,0.000000,0.214286
West Bengal,22.9868,87.8550,310,12,62,236,0.038710,0.200000


In [50]:
temp = temp[['Confirmed','Deaths','Cured','Active','Mortality Rate','Recovery Rate']]
temp.sort_values('Confirmed',ascending=False,inplace = True)
#temp.head()

temp.style\
    .background_gradient(cmap="Blues", subset=['Active','Confirmed'])\
    .background_gradient(cmap="Greens", subset=['Cured', 'Recovery Rate'])\
    .background_gradient(cmap="Reds", subset=['Deaths', 'Mortality Rate'])

,Confirmed,Deaths,Cured,Active,Mortality Rate,Recovery Rate
State/UT,,,,,,
Maharashtra,3651,211,365,3075,0.057792,0.099973
Delhi,1893,42,72,1779,0.022187,0.038035
Madhya Pradesh,1407,70,127,1210,0.049751,0.090263
Gujarat,1376,53,93,1230,0.038517,0.067587
Tamil Nadu,1372,15,365,992,0.010933,0.266035
Rajasthan,1351,11,183,1157,0.008142,0.135455
Uttar Pradesh,969,14,86,869,0.014448,0.088751
Telengana,809,18,186,605,0.022250,0.229913
Andhra Pradesh,603,15,42,546,0.024876,0.069652


In [51]:
temp.columns

Index(['Confirmed', 'Deaths', 'Cured', 'Active', 'Mortality Rate',
       'Recovery Rate'],
      dtype='object')

In [52]:
#Visualization
temp_1 = temp[['Confirmed', 'Deaths', 'Cured']]

temp_1.iplot(kind = 'bar',xTitle= 'State/UT' , yTitle='Numbers of Cases',mode = 'markers+lines',
            title = f'Cases State Wise on {now}')

In [53]:
temp_2 = temp[['Mortality Rate','Recovery Rate']]
               
temp_2.iplot(kind ='scatter',xTitle='State/UT',yTitle='Avrage',title = f'Mortality and Recovery Rate on {now}',
             mode = 'markers', size = 5)

In [54]:
# Date wise data visualization whole country

temp = df_clean.groupby(by = ['Date']).sum()
temp.drop(['Latitude','Longitude','Mortality Rate','Recovery Rate'],axis=1,inplace=True)

temp.tail()

,Confirmed,Deaths,Cured,Active
Date,,,,
2020-04-15,11439,377,1306,9756
2020-04-16,12380,414,1489,10477
2020-04-17,13387,437,1749,11201
2020-04-18,14792,488,2015,12289
2020-04-19,15712,507,2231,12974


In [55]:
temp.iplot(title = 'Covid-19 Growth in India', yTitle='Cases',mode='markers+lines',size=5)

## Top 10 States

In [56]:
top_10 = latest.groupby(by = ['State/UT']).agg({'Confirmed': 'sum', 'Deaths': 'sum', 'Cured' : 'sum', 'Active' : 'sum'})\
                .nlargest(10,['Confirmed','Deaths','Cured','Active'])

top_10

,Confirmed,Deaths,Cured,Active
State/UT,,,,
Maharashtra,3651,211,365,3075
Delhi,1893,42,72,1779
Madhya Pradesh,1407,70,127,1210
Gujarat,1376,53,93,1230
Tamil Nadu,1372,15,365,992
Rajasthan,1351,11,183,1157
Uttar Pradesh,969,14,86,869
Telengana,809,18,186,605
Andhra Pradesh,603,15,42,546


In [57]:
#Creating Figures
plot_c = px.bar(top_10.sort_values('Confirmed') ,x="Confirmed",y = top_10.sort_values('Confirmed').index,
               text='Confirmed', orientation='h', color_discrete_sequence = [cnf])

plot_d = px.bar(top_10.sort_values('Deaths'),x="Deaths",y = top_10.sort_values('Deaths').index,
               text='Deaths', orientation='h', color_discrete_sequence = [dth])

plot_r = px.bar(top_10.sort_values('Cured'),x="Cured",y = top_10.sort_values('Cured').index,
               text='Cured', orientation='h', color_discrete_sequence = [rec])

plot_a = px.bar(top_10.sort_values('Active'),x="Active",y = top_10.sort_values('Active').index,
               text='Active', orientation='h', color_discrete_sequence = [act])


# plot
fig = make_subplots(rows=2, cols=2, shared_xaxes=False, horizontal_spacing=0.14, vertical_spacing=0.08,
                    subplot_titles=('Confirmed cases', 'Deaths reported', 'Recovered', 'Active cases'))

fig.add_trace(plot_c['data'][0],row=1, col=1)
fig.add_trace(plot_d['data'][0],row=1, col=2)
fig.add_trace(plot_r['data'][0],row=2, col=1)
fig.add_trace(plot_a['data'][0],row=2, col=2)

fig.update_layout(height=600 ,title_text="Top 10 States ")


# Cases Rises in State Over Time

https://app.flourish.studio/visualisation/1977187/edit

In [58]:
HTML('<div class="flourish-embed flourish-bar-chart-race" data-src="visualisation/1977187" data-url="https://flo.uri.sh/visualisation/1977187/embed"><script src="https://public.flourish.studio/resources/embed.js"></script></div>')

In [59]:
df_clean.columns

Index(['Date', 'State/UT', 'Latitude', 'Longitude', 'Confirmed', 'Deaths',
       'Cured', 'Active', 'Mortality Rate', 'Recovery Rate'],
      dtype='object')

In [60]:
#Deaths,Cured ,Active Cases Date Wise

col = ['Deaths','Cured','Active']

for i,val in enumerate(col):
    p_df = pd.pivot_table(df_clean,index  = 'Date', values = val, columns ='State/UT').fillna(0).astype('int').reset_index()
    p_df.iplot(x = 'Date' ,title = col[i], xTitle = 'Date',yTitle = 'Cases')

# Geographical Map

In [61]:
latest.columns

Index(['Date', 'State/UT', 'Latitude', 'Longitude', 'Confirmed', 'Deaths',
       'Cured', 'Active', 'Mortality Rate', 'Recovery Rate'],
      dtype='object')

In [62]:
geo_map = folium.Map([20.5937,78.9629],zoom_start=4,tiles ='cartodbpositron' )

'mapboxcontrolroom'


for lat,long,active,deaths,cured,name in zip(latest['Latitude'],latest['Longitude'],\
                                             latest['Active'],latest['Deaths'],\
                                             latest['Cured'],latest['State/UT']):

    folium.CircleMarker([lat,long],radius=active*0.03\
                       ,tooltip = (f'''<strong>name</strong>: {str(name).capitalize()} <br>
                               <strong>Active</strong>: {str(active)}<br>
                               <strong>Deaths</strong>: {str(deaths)}<br>
                               <strong>Cured</strong>: {str(cured)}<br>''')\
                       ,color = 'red',fill_color = 'red',fill_opacity=0.3).add_to(geo_map)

geo_map